In [7]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import geopandas as gpd
import importlib
import models
importlib.reload(models)
from models import TickedRoute, Route, get_conn
pd.set_option('display.expand_frame_repr', False) # display full data in terminal

engine, Session = get_conn()

In [2]:
# Fetch Tick and Route data from DB, organize into geo_data (list of lat/long, name)
with Session() as session:
    statement = session.query(Route).statement
    routes_df = pd.read_sql(statement, con=session.bind)

    tick_statement = session.query(TickedRoute.routeid).statement
    ticks_df = pd.read_sql(tick_statement, con=session.bind)

geo_data = []
ticked_ids = ticks_df['routeid'].tolist()

for idx, route in routes_df.iterrows():
    geo_data.append({'latitude': route['area_latitude'], 'longitude': route['area_longitude'], 'name': route['route_name'], 'ticked': route['routeid'] in ticked_ids})


In [10]:
# Make a map with all 110k routes. save to HTML as it doesnt render well in vs code.

map_center = [geo_data[0]['latitude'], geo_data[0]['longitude']]
route_map = folium.Map(location=map_center, zoom_start=5)
marker_cluster = MarkerCluster().add_to(route_map)

# Add points to the map
for point in geo_data:
    color = 'green' if point['ticked'] else 'blue'
    folium.Marker(
        location=[point['latitude'], point['longitude']],
        tooltip=point['name'],
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)

# Save the map to an HTML file
route_map.save('html_maps/climbing_areas.html')


In [ ]:
# for rendering in VS code, make a new map with a subset of the full data (10000 routes.).

geo_data_small = geo_data[:200]
map_center = [geo_data_small[0]['latitude'], geo_data_small[0]['longitude']]
small_map = folium.Map(location=map_center, width='80%', height=700, zoom_start=5)
small_cluster = MarkerCluster().add_to(small_map)

for point in geo_data_small:
    color = 'green' if point['ticked'] else 'blue'
    folium.Marker(
        location=[point['latitude'], point['longitude']],
        tooltip=point['name'],
        icon=folium.Icon(color=color)
    ).add_to(small_cluster)

small_map

In [ ]:
# read GaiaGPS Trails out of PostGIS, as dataframe
with Session() as session:
    gdf = gpd.read_postgis("SELECT * FROM trails", con=engine, geom_col='geometry')

map = folium.Map(location=[gdf['latitude'].mean(), gdf['longitude'].mean()], width='80%', height=700, zoom_start=5)

for _, row in gdf.iterrows():
    folium.GeoJson(row['geometry'], name=row['title']).add_to(map)

map.save('html_maps/gaia_trails.html')
map